<a href="https://colab.research.google.com/github/wjleece/anthropic-prompts/blob/main/Anthropic_Customer_Service_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 5.5 MB/s eta 0:00:00


In [23]:
import anthropic
import re
from google.colab import userdata

API_KEY = userdata.get('ANTHROPIC_API_KEY')
MODEL_NAME = "claude-3-7-sonnet-latest"
client = anthropic.Anthropic(api_key=API_KEY)

In [3]:
system_prompt = "You are a helpful assistant."

In [4]:
tools = []

In [24]:
def get_completion(prompt: str):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        tools=tools,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

In [6]:
prompt="Yo Claude, wuzzup?"

print(get_completion(prompt))

I'm doing well, thanks for asking. How are you today?


In [30]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
     {
        "name": "add_customer",
        "description": "Adds a new customer to the database. Includes customer ID, name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer, which you automatically will generate any time add_customer is called."
                },
                "name": {
                    "type": "string",
                    "description": "The name of the customer."
                },
                "email": {
                    "type": "string",
                    "description": "The email address of the customer."
                }
            },
            "required": ["customer_id", "name", "email", "phone"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "add_order",
        "description": "Creates an order. Includes order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order, which you automatically will generate any time add_order is called."
                },
                "product": {
                    "type": "string",
                    "description": "The name of the product in the order."
                },
                "quantity": {
                    "type": "integer",
                    "description": "The quantity of the product in the order."
                },
                "price": {
                    "type": "number",
                    "description": "The price of each product in the order."
                },
            },
            "required": ["order_id", "product", "quantity", "price", "status" ]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [26]:
# Global customer storage
customers = {
    "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
    "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
}

# Global order storage
orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }

def get_customer_info(customer_id):
    return customers.get(customer_id, "Customer not found")

def add_customer(name, email, phone):
    new_id = f"C{len(customers) + 1}"  # Simple ID generator like C3, C4, ...
    customers[new_id] = {"name": name, "email": email, "phone": phone}
    return new_id, customers[new_id]

def get_order_details(order_id):
    # Simulated order data
    return orders.get(order_id, "Order not found")

def add_order(product, quantity, price, status):
    new_id = f"O{len(orders) + 1}"  # Simple ID generator like O
    orders[new_id] = {"id": new_id, "product": product, "quantity": quantity, "price": price, "status": status}
    return new_id, orders[new_id]

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

In [27]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "add_customer":
        return add_customer(tool_input["name"], tool_input["email"], tool_input["phone"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "add_order":
        return add_order(tool_input["product"], tool_input["quantity"], tool_input["price"], tool_input["status"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

In [28]:
import json

def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

In [11]:
chatbot_interaction("Can you tell me the email address for customer C1?")


User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you retrieve the customer information for customer ID C1. I'll use the `get_customer_info` function to fetch the details.", type='text'), ToolUseBlock(id='toolu_018jqbYEmP9ncsrjAqo58B8s', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?', type='text')]

Final Response: The email address for customer C1 (John Doe) is john@example.com.

Is there anything else I can help you with?


'The email address for customer C1 (John Doe) is john@example.com.\n\nIs there anything else I can help you with?'

In [12]:
chatbot_interaction("What is the status of order O2?")


User Message: What is the status of order O2?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you retrieve the details of order O2 using the get_order_details function.", type='text'), ToolUseBlock(id='toolu_01JPibQPjrneK5sbrU1XXKyS', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O2"
}

Tool Result:
{
  "id": "O2",
  "product": "Gadget B",
  "quantity": 1,
  "price": 49.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='The status of order O2 is "Processing". \n\nHere are the additional details of the order:\n- Product: Gadget B\n- Quantity: 1\n- Price: $49.99\n\nIs there anything else I can help you with regarding this order?', type='text')]

Final Response: The status of order O2 is "Processing". 

Here are the additional details of the order:
- Product: Gadget B
- Quantity: 1
- Price: $49.99

Is there 

'The status of order O2 is "Processing". \n\nHere are the additional details of the order:\n- Product: Gadget B\n- Quantity: 1\n- Price: $49.99\n\nIs there anything else I can help you with regarding this order?'

In [13]:
chatbot_interaction("Please cancel order O1 for me.")


User Message: Please cancel order O1 for me.

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you cancel the order with ID O1. I'll use the `cancel_order` function to process this request.", type='text'), ToolUseBlock(id='toolu_012vZRV4gw2KiC4zhD9Qu7Tn', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')]

Tool Used: cancel_order
Tool Input:
{
  "order_id": "O1"
}

Tool Result:
true

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='The order O1 has been successfully cancelled. Is there anything else I can help you with today?', type='text')]

Final Response: The order O1 has been successfully cancelled. Is there anything else I can help you with today?


'The order O1 has been successfully cancelled. Is there anything else I can help you with today?'

In [14]:
print(get_order_details("O1"))

{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Shipped'}


In [31]:
chatbot_interaction("Add a new customer: Bill Leece, bill.leece@example.com, 123-444-7777")


User Message: Add a new customer: Bill Leece, bill.leece@example.com, 123-444-7777

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you add a new customer to the database. I have all the required information to create the customer profile:", type='text'), ToolUseBlock(id='toolu_011GgQcHapLhpFeycoX3VsXG', input={'customer_id': 'CUST-123', 'name': 'Bill Leece', 'email': 'bill.leece@example.com', 'phone': '123-444-7777'}, name='add_customer', type='tool_use')]

Tool Used: add_customer
Tool Input:
{
  "customer_id": "CUST-123",
  "name": "Bill Leece",
  "email": "bill.leece@example.com",
  "phone": "123-444-7777"
}

Tool Result:
[
  "C3",
  {
    "name": "Bill Leece",
    "email": "bill.leece@example.com",
    "phone": "123-444-7777"
  }
]

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='Great! Bill Leece has been successfully added to the database with the following details:\n- Customer ID: C3\n- Name: Bill Leece\n- Em

'Great! Bill Leece has been successfully added to the database with the following details:\n- Customer ID: C3\n- Name: Bill Leece\n- Email: bill.leece@example.com\n- Phone: 123-444-7777\n\nThe customer information has been stored in the system. Is there anything else you would like to do regarding this customer?'

In [16]:
customers

{'C1': {'name': 'John Doe',
  'email': 'john@example.com',
  'phone': '123-456-7890'},
 'C2': {'name': 'Jane Smith',
  'email': 'jane@example.com',
  'phone': '987-654-3210'},
 'C3': {'name': 'Bill Leece',
  'email': 'bill.leece@example.com',
  'phone': '123-456-7777'}}

In [32]:
chatbot_interaction("Create a new order for: Perplexinator, 12, $59.99, Shipped")


User Message: Create a new order for: Perplexinator, 12, $59.99, Shipped

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll help you create a new order for the Perplexinator product. I have most of the details needed to create the order, but I'll need to generate an order ID automatically as part of the process.", type='text'), ToolUseBlock(id='toolu_015fmCJYjemRUJRiCZRcpcRU', input={'order_id': 'ORD-2023-11579', 'product': 'Perplexinator', 'quantity': 12, 'price': 59.99, 'status': 'Shipped'}, name='add_order', type='tool_use')]

Tool Used: add_order
Tool Input:
{
  "order_id": "ORD-2023-11579",
  "product": "Perplexinator",
  "quantity": 12,
  "price": 59.99,
  "status": "Shipped"
}

Tool Result:
[
  "O3",
  {
    "id": "O3",
    "product": "Perplexinator",
    "quantity": 12,
    "price": 59.99,
    "status": "Shipped"
  }
]

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='Your new order has been successfully created! He

'Your new order has been successfully created! Here are the details:\n\n- Order ID: O3\n- Product: Perplexinator\n- Quantity: 12\n- Price: $59.99 each\n- Status: Shipped\n\nThe order has been entered into the system and is marked as shipped. Is there anything else you would like to know about this order or would you like to place another order?'

In [33]:
orders

{'O1': {'id': 'O1',
  'product': 'Widget A',
  'quantity': 2,
  'price': 19.99,
  'status': 'Shipped'},
 'O2': {'id': 'O2',
  'product': 'Gadget B',
  'quantity': 1,
  'price': 49.99,
  'status': 'Processing'},
 'O3': {'id': 'O3',
  'product': 'Perplexinator',
  'quantity': 12,
  'price': 59.99,
  'status': 'Shipped'}}